# Building your Deep Neural Network: Step by Step

Import Statements

In [68]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases import *
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline

Initialize parameters of the 2-layer neural network

In [69]:
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)

    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros(shape=(n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros(shape=(n_y, 1))
    
    

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

In [71]:
parameters = initialize_parameters(2,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.00865408 -0.02301539]]
b2 = [[0.]]


Initialize parameters for an L-layer Neural Network

In [72]:
def initialize_parameters_deep(layer_dims):

    parameters = {}
    
    L = len(layer_dims)           

    for l in range(1, L):
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
       
        
        
    return parameters

In [74]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01659802  0.00742044 -0.00191836 -0.00887629 -0.00747158]
 [ 0.01692455  0.00050808 -0.00636996  0.00190915  0.02100255]
 [ 0.00120159  0.00617203  0.0030017  -0.0035225  -0.01142518]
 [-0.00349343 -0.00208894  0.00586623  0.00838983  0.00931102]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[ 0.00285587  0.00885141 -0.00754398  0.01252868]
 [ 0.0051293  -0.00298093  0.00488518 -0.00075572]
 [ 0.01131629  0.01519817  0.02185575 -0.01396496]]
b2 = [[0.]
 [0.]
 [0.]]




Linear function for forward propogation

In [76]:
def linear_forward(A, W, b):

    Z = np.dot(W, A) + b

    
    #cache stores A,b,W for backward propogation
    cache = (A, W, b)
    
    return Z, cache

In [78]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]




Forward propogation of linear activation layer

In [79]:


def linear_activation_forward(A_prev, W, b, activation):

    
    if activation == "sigmoid":
   
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)

    
    elif activation == "relu":

        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)

    
    
    cache = (linear_cache, activation_cache)

    return A, cache

In [80]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


In [81]:
# L-layer model(forward propogation)

def L_model_forward(X, parameters):

    
    caches = []
    A = X
    # number of layers in the neural network
    L = len(parameters) // 2                
    
    
    for l in range(1, L):
        A_prev = A 
       
        A, cache = linear_activation_forward(A_prev, parameters['W' +str(l)], parameters['b'+str(l)],activation= 'relu')
        caches.append(cache)
     


    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)],activation= 'sigmoid')
    caches.append(cache)

    
            
    return AL, caches

In [83]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3





Cost Computation

In [84]:


def compute_cost(AL, Y):

    m = Y.shape[1]

    
    cost = (-1 / m) * np.sum(np.multiply(Y, np.log(AL)) + np.multiply(1 - Y, np.log(1 - AL)))
    
    
    cost = np.squeeze(cost)

    
    return cost

In [86]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694





Backword propogation

In [87]:


def linear_backward(dZ, cache):

    A_prev, W, b = cache
    m = A_prev.shape[1]


    dW = (1/m) * np.dot(dZ,A_prev.T)
    db = (1/m) * np.sum(dZ, axis = 1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)
   
    
    
    return dA_prev, dW, db

In [88]:
#test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


In [89]:
#Backward propogation of linear activation layer

def linear_activation_backward(dA, cache, activation):

    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
       
        dZ = relu_backward(dA, activation_cache)

        
    elif activation == "sigmoid":
 
        dZ = sigmoid_backward(dA, activation_cache)

    
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

In [90]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


In [91]:
# L-layer model(backward propogation)

def L_model_backward(AL, Y, caches):

    
    grads = {}
    # the number of layers
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    

    current_cache = caches[-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL,current_cache,'sigmoid')

    

    for l in reversed(range(L-1)):
        
        # lth layer
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)],current_cache,'relu')
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] =dW_temp
        grads["db" + str(l + 1)] = db_temp
      

    return grads

In [92]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]]


In [93]:
# Gradient descent

def update_parameters(parameters, grads, learning_rate):

    

    L = len(parameters) // 2


    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]

        
    return parameters

In [95]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]
